# StarGal Full Run

Fully detailed run to compute photometric redshift of a,mixed catalog (star+galaxy)

In [1]:
import os
from importlib import reload
import lephare as lp; reload(lp)
import numpy as np
from matplotlib import pylab as plt

%matplotlib inline

base_dir = os.path.abspath(os.path.join(os.getcwd(), '../..')) #change to your lephare base_dir
print(base_dir)

LEPHAREDIR is being set to the default cache directory:
/home/hallouin/.cache/lephare/data
More than 1Gb may be written there.
LEPHAREWORK is being set to the default cache directory:
/home/hallouin/.cache/lephare/work
Default work cache is already linked. 
This is linked to the run directory:
/home/hallouin/.cache/lephare/runs/20251013T115331
User defined LEPHAREDIR is set. Code runs depend on all required
auxiliary data being present at /home/hallouin/.cache/lephare/data.
User defined LEPHAREWORK is set. All intermediate files will be written to:
 /home/hallouin/.cache/lephare/work
/home/hallouin/Documents/t_hall/lephare


In [2]:
config = lp.default_stargal_config.copy()

config.update({"VERBOSE": "NO", "Z_STEP": "0.01,0.,2.",})

config = lp.default_stargal_config.copy()

## Prepare MagGal libraries

### DC1 - Make filter library

In [3]:
##### Create filter library #####
filter_path = os.path.join(base_dir, 'training_stats/simulation_catalogs/buzzard_base/filt') 
print(filter_path)
filter_rep = lp.keyword("FILTER_REP", filter_path)
filter_list = lp.keyword("FILTER_LIST",
"BuzzardLSSTu.res,BuzzardLSSTg.res,BuzzardLSSTr.res,\
     BuzzardLSSTi.res,BuzzardLSSTz.res,BuzzardLSSTy4.res")

filterLib = lp.Filter(config_keymap={"FILTER_REP":filter_rep,
                                   "FILTER_LIST":filter_list,
                                   "TRANS_TYPE":lp.keyword("TRANS_TYPE", "0"),
                                   "FILTER_CALIB":lp.keyword("FILTER_CALIB", "0"), #"0,0,0,0,0,0"
                                   "FILTER_FILE":lp.keyword("FILTER_FILE", "photozdc1")
                                     })

filterLib.run()

/home/hallouin/Documents/t_hall/lephare/training_stats/simulation_catalogs/buzzard_base/filt
# NAME                        IDENT      Lbda_mean Lbeff(Vega)       FWHM     AB-cor    TG-cor      VEGA M_sun(AB)   CALIB      Lb_eff    Fac_corr
BuzzardLSSTu.res              1             0.3733      0.3800      0.0530    0.5173   -0.3186  -20.7758    6.2286       0      0.3711      1.0000
BuzzardLSSTg.res              2             0.4799      0.4700      0.1453   -0.0893   -0.2698  -20.6973    5.1149       0      0.4719      1.0000
BuzzardLSSTr.res              3             0.6221      0.6133      0.1391    0.1458    0.2557  -21.5085    4.6531       0      0.6165      1.0000
BuzzardLSSTi.res              4             0.7541      0.7475      0.1302    0.3662    0.5795  -22.1521    4.5371       0      0.7499      1.0000
BuzzardLSSTz.res              5             0.8682      0.8659      0.1011    0.5171    0.7616  -22.6138    4.5165       0      0.8658      1.0000
BuzzardLSSTy4.res        

### DC1- Make SED library

In [4]:
##### Create SED library #####

SED_path = os.path.join(base_dir, 'training_stats/simulation_catalogs/buzzard_base/lib_bin/updated_Buzzard_SEDs')

# SED_list = sorted([
#     f for f in os.listdir(SED_path)
#     if os.path.isfile(os.path.join(SED_path, f)) and f.endswith(".sed")
# ])

SED_list = [f'kmeansbuzzard_{i}.sed' for i in range(0,100)]

#save to file.list because we must do it like this
SED_list_path = os.path.join(SED_path, "updated_Buzzard_SEDs.list")
with open(SED_list_path, "w") as f:
    for sed_file in SED_list:
        f.write("Buzzard/"+sed_file + "\n")

SED_rep = lp.keyword("SED_REP", SED_path)
print(SED_rep)

###add SED to cahe file because ...###
import shutil

#Lephare cache file
LEPHARE_SED_GAL_PATH = os.path.expanduser("~/.cache/lephare/data/sed/GAL/Buzzard")

#Create cache file in case
os.makedirs(LEPHARE_SED_GAL_PATH, exist_ok=True)

# Cpypast all file to cache
for sed_file in SED_list + ["updated_Buzzard_SEDs.list"]:
    src = os.path.join(SED_path, sed_file)
    dst = os.path.join(LEPHARE_SED_GAL_PATH, sed_file)
    shutil.copyfile(src, dst)

print(f"past {len(SED_list)} file.sed and file.list in {LEPHARE_SED_GAL_PATH}")



SED_list_path = f"{base_dir}/training_stats/simulation_catalogs/buzzard_base/lib_bin/updated_Buzzard_SEDs/updated_Buzzard_SEDs.list"
sedLib = lp.Sedtolib(config_keymap={
    "SED_REP": SED_rep,
    "GAL_SED": lp.keyword("GAL_SED", SED_list_path),
    "GAL_FSCALE": lp.keyword("GAL_FSCALE", "1.0"),
    "GAL_LIB": lp.keyword("GAL_LIB", 'buzzard_SEDs')},)
sedLib.run(typ="G")

(SED_REP, /home/hallouin/Documents/t_hall/lephare/training_stats/simulation_catalogs/buzzard_base/lib_bin/updated_Buzzard_SEDs)
past 100 file.sed and file.list in /home/hallouin/.cache/lephare/data/sed/GAL/Buzzard
#######################################
# It s translating SEDs to binary library #
# with the following options :           
# Config file     : 
# Library type     : GAL
# GAL_SED    :/home/hallouin/Documents/t_hall/lephare/training_stats/simulation_catalogs/buzzard_base/lib_bin/updated_Buzzard_SEDs/updated_Buzzard_SEDs.list
# GAL_LIB    :buzzard_SEDs
# GAL_LIB doc:/home/hallouin/.cache/lephare/work/lib_bin/buzzard_SEDs.doc
# GAL_LIB phys:/home/hallouin/.cache/lephare/work/lib_bin/buzzard_SEDs.phys
# SEL_AGE    :none
# GAL_FSCALE :1.0000
# AGE_RANGE   -999.0000 -999.0000
#######################################
Number of SED in the list 100


### STARS - Make filter library

In [5]:
filter_path = os.path.join(base_dir, 'training_stats/simulation_catalogs/DES/DES_STARCAT/WORK_COMPLETE2/filt') #paste your relative filter path
print(filter_path)
filter_rep = lp.keyword("FILTER_REP", filter_path)
filter_list = lp.keyword("FILTER_LIST",
                         "DES_filter_g.res,DES_filter_r.res,\
                         DES_filter_i.res,DES_filter_z.res,DES_filter_Y.res") #edit your filter list
#crash if add comma

filterLib2 = lp.Filter(config_keymap={"FILTER_REP":filter_rep,
                                   "FILTER_LIST":filter_list,
                                   "TRANS_TYPE":lp.keyword("TRANS_TYPE", "0"),
                                   "FILTER_CALIB":lp.keyword("FILTER_CALIB", "0"),
                                   "FILTER_FILE":lp.keyword("FILTER_FILE", "photozDES")})


filterLib2.run()

/home/hallouin/Documents/t_hall/lephare/training_stats/simulation_catalogs/DES/DES_STARCAT/WORK_COMPLETE2/filt
# NAME                        IDENT      Lbda_mean Lbeff(Vega)       FWHM     AB-cor    TG-cor      VEGA M_sun(AB)   CALIB      Lb_eff    Fac_corr
DES_filter_g.res              1             0.4827      0.4735      0.1299   -0.0867   -0.2558  -20.7147    5.0926       0      0.4753      1.0000
DES_filter_r.res              2             0.6432      0.6343      0.1484    0.1835    0.3154  -21.6187    4.6235       0      0.6375      1.0000
DES_filter_i.res              3             0.7827      0.7749      0.1482    0.4149    0.6384  -22.2807    4.5296       0      0.7779      1.0000
DES_filter_z.res              4             0.9179      0.9139      0.1471    0.5235    0.7629  -22.7383    4.5124       0      0.9139      1.0000
DES_filter_Y.res              5             0.9900      0.9880      0.0661    0.5722    0.7890  -22.9560    4.5088       0      0.9886      1.0000


### STARS - Make SED library

In [6]:
#### Create SED library #####
#ready in lephare data
#replace by STAR_MOD.list if you want full list
SED_list_path = f"{base_dir}/training_stats/simulation_catalogs/DES/DES_STARCAT/WORK_COMPLETE2/lib_bin/DES_STAR.list"
sedLib = lp.Sedtolib(config_keymap={
    "STAR_SED": lp.keyword("STAR_SED", SED_list_path),
    "STAR_FSCALE": lp.keyword("STAR_FSCALE", "3.0e-9"),
    "STAR_LIB": lp.keyword("STAR_LIB", 'PICKLES_SEDs')},)
sedLib.run(typ="S")

#######################################
# It s translating SEDs to binary library #
# with the following options :           
# Config file     : 
# Library type     : STAR
# STAR_SED    :/home/hallouin/Documents/t_hall/lephare/training_stats/simulation_catalogs/DES/DES_STARCAT/WORK_COMPLETE2/lib_bin/DES_STAR.list
# STAR_LIB    :PICKLES_SEDs
# STAR_LIB doc:/home/hallouin/.cache/lephare/work/lib_bin/PICKLES_SEDs.doc
# STAR_FSCALE :0.0000
#######################################
Number of SED in the list 42


## Run MagGal

Create a magnitude library from the SED and filter libraries.
To compare properly our available libraries, we need to run 4 times mag_gal. 

In [7]:
maglib = lp.MagGal(config_keymap=lp.all_types_to_keymap(config))

### buzzard in LSST filters (for DC1, only gal)

In [8]:
maglib.run(
    typ="G",
    gal_lib_in="buzzard_SEDs",
    gal_lib_out="buzzard_LSST",
    filter_file="photozdc1",
    magtype="AB",
    mod_extinc="0",
    extinc_law="calzetti.dat",
    em_dispersion="0",
    lib_ascii="YES",
    z_step="0.01,0.,2."
)


#######################################
# It s computing the SYNTHETIC MAGNITUDES #
# For Gal/QSO libraries with these OPTIONS #
# with the following options :           
# Config file     : 
# Filter file     : photozdc1
# Magnitude type     : AB
# GAL_LIB_IN    :/home/hallouin/.cache/lephare/work/lib_bin/buzzard_SEDs(.doc & .bin)
# GAL_LIB_OUT   :/home/hallouin/.cache/lephare/work/lib_mag/buzzard_LSST(.doc & .bin)
# Z_STEP   :0.0100 0.0000 2.0000
# COSMOLOGY   :70.0000,0.3000,0.7000
# EXTINC_LAW   :calzetti.dat 
# MOD_EXTINC   :0 0 
# EB_V   :0.0000 
# EM_LINES   NO
# EM_DISPERSION   1.0000,
# LIB_ASCII   YES
# CREATION_DATE Thu Oct 23 10:57:38 2025
#############################################


### buzzard in DES filters (for DESstars, star-gal)

In [9]:
maglib.run(
    typ="G",
    gal_lib_in="buzzard_SEDs",
    gal_lib_out="buzzard_DES",
    filter_file="photozDES",
    magtype="AB",
    mod_extinc="0",
    extinc_law="calzetti.dat",
    em_dispersion="0",
    lib_ascii="YES",
    z_step="0.01,0.,2."
)

#######################################
# It s computing the SYNTHETIC MAGNITUDES #
# For Gal/QSO libraries with these OPTIONS #
# with the following options :           
# Config file     : 
# Filter file     : photozDES
# Magnitude type     : AB
# GAL_LIB_IN    :/home/hallouin/.cache/lephare/work/lib_bin/buzzard_SEDs(.doc & .bin)
# GAL_LIB_OUT   :/home/hallouin/.cache/lephare/work/lib_mag/buzzard_DES(.doc & .bin)
# Z_STEP   :0.0100 0.0000 2.0000
# COSMOLOGY   :70.0000,0.3000,0.7000
# EXTINC_LAW   :calzetti.dat 
# MOD_EXTINC   :0 0 
# EB_V   :0.0000 
# EM_LINES   NO
# EM_DISPERSION   1.0000,
# LIB_ASCII   YES
# CREATION_DATE Thu Oct 23 10:57:40 2025
#############################################


### PICKLES in DES filters (for DESstars, only star)

In [10]:
maglib.run(
    typ="S",
    star_lib_in="PICKLES_SEDs",
    star_lib_out="PICKLES_DES",
    filter_file="photozDES",
    magtype="AB",
    mod_extinc="0",
    extinc_law="calzetti.dat",
    em_dispersion="0",
    lib_ascii="YES",
    z_step="0.0,0,0"
)

#######################################
# It s computing the SYNTHETIC MAGNITUDES #
# For Gal/QSO libraries with these OPTIONS #
# with the following options :           
# Config file     : 
# Filter file     : photozDES
# Magnitude type     : AB
# COSMOLOGY   :70.0000,0.3000,0.7000
# STAR_LIB_IN    :/home/hallouin/.cache/lephare/work/lib_bin/PICKLES_SEDs(.doc & .bin)
# STAR_LIB_OUT   :/home/hallouin/.cache/lephare/work/lib_mag/PICKLES_DES(.doc & .bin)
# LIB_ASCII   YES
# CREATION_DATE Thu Oct 23 10:57:41 2025
#############################################


### PICKLES in LSST filters (for DC1, star-gal)

In [11]:
maglib.run(
    typ="S",
    star_lib_in="PICKLES_SEDs",
    star_lib_out="PICKLES_LSST",
    filter_file="photozdc1",
    magtype="AB",
    mod_extinc="0",
    extinc_law="calzetti.dat",
    em_dispersion="0",
    lib_ascii="YES",
    z_step="0.0,0,0"
)

#######################################
# It s computing the SYNTHETIC MAGNITUDES #
# For Gal/QSO libraries with these OPTIONS #
# with the following options :           
# Config file     : 
# Filter file     : photozdc1
# Magnitude type     : AB
# COSMOLOGY   :70.0000,0.3000,0.7000
# STAR_LIB_IN    :/home/hallouin/.cache/lephare/work/lib_bin/PICKLES_SEDs(.doc & .bin)
# STAR_LIB_OUT   :/home/hallouin/.cache/lephare/work/lib_mag/PICKLES_LSST(.doc & .bin)
# LIB_ASCII   YES
# CREATION_DATE Thu Oct 23 10:57:41 2025
#############################################


## Run photoz

Finally run the photoz. We try it for two different photometry catalogs: calibrated star from DES catalog (in DES filters) and DC1 simulation catalog (in LSST filters).

### DC1


Initialization

In [ ]:
config.update(
    {
        "ZPHOTLIB": "buzzard_LSST,PICKLES_LSST",
        "SPEC_OUT": "NO",
        "CAT_IN": f"{base_dir}/training_stats/simulation_catalogs/buzzard_base/Shuffle_Buzzard_training_file.dat",
        "INP_TYPE": "M",
        "CAT_TYPE": "LONG",
        "CAT_MAG": "AB",
        "PARA_OUT": f"{base_dir}/training_stats/simulation_catalogs/star_gal/stargal_output.para",
        "CAT_OUT": f"{base_dir}/training_stats/simulation_catalogs/star_gal/DC1_Buzzard_PICKLES_LSST.out",
        "ZPHOTLIB": "buzzard_LSST,PICKLES_LSST",
        "AUTO_ADAPT": "NO",
        "STAR_PDF_OUT": f"{base_dir}/training_stats/simulation_catalogs/star_gal/PDFs/DC1_Buzzard_PICKLES_LSST_kk",
        "PDZ_OUT": f"{base_dir}/training_stats/simulation_catalogs/star_gal/PDZs/DC1_Buzzard_PICKLES_LSST_piton",
        "PDZ_TYPE": "MIN_ZG",
        "CAT_FMT": "MMEE",
        "ERR_SCALE": 0,
        "SPEC_OUT": "NO",
        "CHI2_OUT": "NO"
    }
)

zphota = lp.PhotoZ(lp.all_types_to_keymap(config))

####################################### 
# PHOTOMETRIC REDSHIFT with OPTIONS   # 
# Config file            : 
# CAT_IN                 : /home/hallouin/Documents/t_hall/lephare/training_stats/simulation_catalogs/buzzard_base/Shuffle_Buzzard_training_file.dat
# CAT_OUT                : /home/hallouin/Documents/t_hall/lephare/training_stats/simulation_catalogs/star_gal/DC1_Buzzard_PICKLES_LSST.out
# CAT_LINES              : 0 1000000000
# PARA_OUT               : /home/hallouin/Documents/t_hall/lephare/training_stats/simulation_catalogs/star_gal/stargal_output.para
# INP_TYPE               : M
# CAT_FMT[0:MEME 1:MMEE] : 1
# CAT_MAG                : AB
# ZPHOTLIB               : buzzard_LSST PICKLES_LSST 
# FIR_LIB                : 
# FIR_LMIN               : 7.000000
# FIR_CONT               : -1.000000
# FIR_SCALE              : -1.000000
# FIR_FREESCALE          : YES
# FIR_SUBSTELLAR         : NO
# ERR_SCALE              : 0.000000 
# ERR_FACTOR             : 1.000000 
# GLB_CONTEXT  

Load catalog

In [13]:
cat = np.loadtxt(f"{base_dir}/training_stats/simulation_catalogs/buzzard_base/Shuffle_Buzzard_training_file.dat")
id = cat[:, 0]
mag = cat[:, 1:7]
emag = cat[:, 7:13]
context = cat[:, 13]
zspec = cat[:, 14]
print("Check format with context and zspec :", context, zspec)

Check format with context and zspec : [0. 0. 0. ... 0. 0. 0.] [0.21434 0.30069 0.36735 ... 1.62676 0.38128 0.62006]


Turn rows into lephare onsources

In [14]:
srclist = []
n = 5
for i in range(n):
    oneObj = lp.onesource(i, zphota.gridz)
    oneObj.readsource(str(id[i]), mag[i, :], emag[i, :], int(context[i]), zspec[i], " ")
    zphota.prep_data(oneObj)
    srclist.append(oneObj)
print("Sources with a spec-z: ", len(srclist))

Sources with a spec-z:  5


In [15]:
zphota.run_photoz(srclist[:n], [])

# photz.run_photoz(photozlist, a0) #if autao adapt on

10004.2056
9572.2619
9729.0498
8223.2310
7621.7029
6604.7596
5050.3638
5180.5638
4454.6349
5043.8312
3641.9977
3695.9100
3130.8475
2420.4090
3071.7041
1742.2552
1036.4407
748.4017
561.6734
285.1812
425.7390
173.6720
70.5638
236.8811
52.0607
103.8085
58.4322
338.8907
1893.2047
1392.2092
643.0191
1355.1183
1865.7928
3653.8185
6574.3338
8806.9563
10077.5848
13802.6148
15833.5363
17921.4430
9596.5389
8703.6427
157421.7494
154273.6851
154171.7897
143328.0861
134604.1945
123937.0654
99833.2618
89399.9185
83253.3693
82847.0838
72604.9701
65880.4472
56981.8752
44609.8546
55794.6048
34329.3679
23494.2105
17365.2031
13091.5216
9694.5900
11016.4031
9312.9181
6759.9858
14317.3025
5202.9567
8571.3158
7353.4990
16585.7664
40571.0951
34247.9949
21043.7205
31283.6971
39057.7654
56634.8545
76763.7823
92647.8411
100728.5035
123499.8265
140453.3580
159616.7047
155363.7279
146568.7603
288.3274
278.2100
284.9497
247.2687
235.9067
210.0386
173.4571
184.2311
161.7021
184.0466
135.2752
140.5265
124.2450
103.3

## Turn output in fits

In [16]:
t = zphota.build_output_tables(srclist[:n], para_out=f"{base_dir}/training_stats/simulation_catalogs/star_gal/stargal_output.para", filename="DESstars_Buzzard_PICKLES_DES.fits")

In [17]:
t[:5]

IDENT,Z_BEST,Z_MED,Z_MODE,CHI_BEST,MOD_BEST,SCALE_BEST,NBAND_USED,Z_SEC,CHI_SEC,MOD_STAR,CHI_STAR,MAG_OBS(),ERR_MAG_OBS(),ZSPEC,MIN_ZG
str8,float64,float64,float64,float64,int64,float64,int64,float64,float64,int64,float64,float64[6],float64[6],float64,float64[201]
2996.0,0.18052984371447872,0.18606492273305222,0.1798440545698019,1.7910833720269452,47,0.004132517808212152,6,-99.9,-99.0,25,52.06068469263381,25.68790000000002 .. 23.51890000000001,0.14 .. 0.0645,0.21434,0.0 .. 3.4202513310656275e-81
87746.0,0.2879387716544945,0.2896291890240811,0.2899017552992538,67.65098683332288,63,0.28073301028947534,6,-99.9,-99.0,25,5202.956720081948,21.993600000000022 .. 20.097500000000004,0.0079 .. 0.0057,0.30069,0.0 .. 0.0
258981.0,0.4507572206496423,0.39648714441152777,0.4215599022668453,1.5933671889567935,82,0.004076845349088743,6,-99.9,-99.0,27,5.820303976838677,26.63150000000001 .. 25.633300000000006,0.3941 .. 0.42610000000000015,0.36735,0.0 .. 0.0004608082170564103
403445.0,1.4949860718500445,1.4575936414208281,1.522774439962626,0.13258772887476508,38,0.1876651020767542,6,0.07,38.0,26,7.787515991319223,27.859600000000004 .. 25.491300000000027,0.7765 .. 0.4666,1.2211,0.0 .. 0.5414526859038372
311170.0,-99.9,-99.9,0.0,1000000000.0,-99,0.0,6,-99.9,-99.0,6,150.05874960441406,99.0 .. 25.639200000000017,2.5024 .. 0.2308,1.06301,0.0 .. 0.0


In [18]:
import time

zphota.write_outputs(srclist[:n], int(time.time()))

In [ ]:
# This created the output ascii file specified in the config CAT_OUT parameter
!ls -al zphota.out